# <center><h1>Companhia Aberta Demonstrativo Financeiro</h1></center>

# <h2>Load Libraries</h2>

In [4]:
import pandas as pd
import numpy as np

import sys

# <h2>Import Data</h2>

## <h3>Load .csv</h3>

In [5]:
cia_aberta_df = pd.read_csv(r'dfp_cia_aberta_BPA_con_2020.csv', encoding='ISO-8859-1', sep=";")    # leio o .csv com os dados de todas as companhias abertas

## <h3>Select Company</h3>

In [6]:
df = cia_aberta_df[cia_aberta_df['CNPJ_CIA'] == '97.837.181/0001-47'].copy()    # (seleciono somente as linhas relativas a uma companhia de interesse)
df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']                                         # seleciono somente o último ou penúltimo exercício
df = df[['CD_CONTA','DS_CONTA','VL_CONTA']]
df.reset_index(inplace=True, drop=True)

In [7]:
df

,CD_CONTA,DS_CONTA,VL_CONTA
0,1,Ativo Total,11498520.0
1,1.01,Ativo Circulante,4220022.0
2,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,1.01.02,Aplicações Financeiras,0.0
4,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...
71,1.02.04.02.07,Goodwill na aquisição da Caetex Florestal,8767.0
72,1.02.04.02.08,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,1.02.04.02.09,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Subset DataFrame (only for testing purposes)</h4>

In [8]:
df = df.iloc[np.r_[0:7, 24:31],:]

In [9]:
df

,CD_CONTA,DS_CONTA,VL_CONTA
0,1,Ativo Total,11498520.0
1,1.01,Ativo Circulante,4220022.0
2,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,1.01.02,Aplicações Financeiras,0.0
4,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.01.02.01.01,Títulos para Negociação,0.0
6,1.01.02.01.02,Títulos Designados a Valor Justo,0.0
24,1.02,Ativo Não Circulante,7278498.0
25,1.02.01,Ativo Realizável a Longo Prazo,2071636.0
26,1.02.01.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0


# <h2>Wrangling</h2>

## <h3>Number of Steps</h3>

In [10]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_len = [len(lst) for lst in cd_conta_split]
num_levels = max(cd_conta_len)

In [11]:
num_levels

5

## <h3>ROUND 1</h3>

In [12]:
round=1

In [13]:
df

,CD_CONTA,DS_CONTA,VL_CONTA
0,1,Ativo Total,11498520.0
1,1.01,Ativo Circulante,4220022.0
2,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,1.01.02,Aplicações Financeiras,0.0
4,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.01.02.01.01,Títulos para Negociação,0.0
6,1.01.02.01.02,Títulos Designados a Valor Justo,0.0
24,1.02,Ativo Não Circulante,7278498.0
25,1.02.01,Ativo Realizável a Longo Prazo,2071636.0
26,1.02.01.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0


### <h4>Join</h4>

In [14]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with only one element:

In [15]:
cd_conta_len = [len(row) for row in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]        # round = 1
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [16]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA,VL_CONTA
0,1,Ativo Total,11498520.0


Join:

In [17]:
ds_conta_1 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [18]:
ds_conta_1 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [19]:
if pd.Series(ds_conta_1).equals(df.DS_CONTA):
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_1', value=ds_conta_1)

In [20]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1,Ativo Total,Ativo Total,11498520.0
1,1.01,Ativo Total,Ativo Circulante,4220022.0
2,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
3,1.01.02,Ativo Total,Aplicações Financeiras,0.0
4,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
6,1.01.02.01.02,Ativo Total,Títulos Designados a Valor Justo,0.0
24,1.02,Ativo Total,Ativo Não Circulante,7278498.0
25,1.02.01,Ativo Total,Ativo Realizável a Longo Prazo,2071636.0
26,1.02.01.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0


### <h4>Drop rows</h4>

Select rows with only one element:

In [21]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1,Ativo Total,Ativo Total,11498520.0


Check if there is at least another CD_CONTA starting with '1'

In [22]:
for idx in idx_list:
    mask = [True]*len(ds_conta_1)
    mask[idx] = False
    if any(row[:round] == cd_conta_joincol[idx] for row in cd_conta_split[mask]):
        df = df.drop(idx)
    else:
        df.loc[idx, 'CD_CONTA'] = df.loc[idx, 'CD_CONTA'] + '.00'

df.reset_index(inplace=True, drop=True)

In [23]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,4220022.0
1,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
2,1.01.02,Ativo Total,Aplicações Financeiras,0.0
3,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
5,1.01.02.01.02,Ativo Total,Títulos Designados a Valor Justo,0.0
6,1.02,Ativo Total,Ativo Não Circulante,7278498.0
7,1.02.01,Ativo Total,Ativo Realizável a Longo Prazo,2071636.0
8,1.02.01.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,1.02.01.01.01,Ativo Total,Títulos Designados a Valor Justo,0.0


## <h3>ROUND 2</h3>

In [24]:
round=2

In [25]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,4220022.0
1,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
2,1.01.02,Ativo Total,Aplicações Financeiras,0.0
3,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
5,1.01.02.01.02,Ativo Total,Títulos Designados a Valor Justo,0.0
6,1.02,Ativo Total,Ativo Não Circulante,7278498.0
7,1.02.01,Ativo Total,Ativo Realizável a Longo Prazo,2071636.0
8,1.02.01.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,1.02.01.01.01,Ativo Total,Títulos Designados a Valor Justo,0.0


### <h4>Join</h4>

In [26]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with only 2 elements:

In [27]:
cd_conta_len = [len(lst) for lst in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [28]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,4220022.0
6,1.02,Ativo Total,Ativo Não Circulante,7278498.0


Join:

In [29]:
ds_conta_2 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [30]:
ds_conta_2 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [31]:
if pd.Series(ds_conta_2).equals(df.DS_CONTA):
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_2', value=ds_conta_2)

In [32]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,Ativo Circulante,4220022.0
1,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
2,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
3,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
5,1.01.02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
6,1.02,Ativo Total,Ativo Não Circulante,Ativo Não Circulante,7278498.0
7,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,2071636.0
8,1.02.01.01,Ativo Total,Ativo Não Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Títulos Designados a Valor Justo,0.0


### <h4>Drop rows</h4>

Select rows with only 2 elements:

In [33]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,Ativo Circulante,4220022.0
6,1.02,Ativo Total,Ativo Não Circulante,Ativo Não Circulante,7278498.0


Check if there is at least another CD_CONTA starting with '1.01' or '1.02':

In [34]:
for idx in idx_list:
    mask = [True]*len(ds_conta_2)
    mask[idx] = False
    if any(row[:round] == cd_conta_joincol[idx] for row in cd_conta_split[mask]):
        df = df.drop(idx)
    else:
        df.loc[idx, 'CD_CONTA'] = df.loc[idx, 'CD_CONTA'] + '.00'

df.reset_index(inplace=True, drop=True)

In [35]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
2,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
3,1.01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
4,1.01.02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
5,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,2071636.0
6,1.02.01.01,Ativo Total,Ativo Não Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
7,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Títulos Designados a Valor Justo,0.0
8,1.02.01.02,Ativo Total,Ativo Não Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,1.02.01.03,Ativo Total,Ativo Não Circulante,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0


## <h3>ROUND 3</h3>

In [36]:
round=3

In [37]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
2,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
3,1.01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
4,1.01.02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
5,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,2071636.0
6,1.02.01.01,Ativo Total,Ativo Não Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
7,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Títulos Designados a Valor Justo,0.0
8,1.02.01.02,Ativo Total,Ativo Não Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,1.02.01.03,Ativo Total,Ativo Não Circulante,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0


### <h4>Join</h4>

In [38]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with only 3 elements:

In [39]:
cd_conta_len = [len(lst) for lst in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [40]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
5,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,2071636.0


Join:

In [41]:
ds_conta_3 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [42]:
ds_conta_3 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [43]:
if pd.Series(ds_conta_3).equals(df.DS_CONTA):
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_3', value=ds_conta_3)

In [44]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras,0.0
2,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
3,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos para Negociação,0.0
4,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos Designados a Valor Justo,0.0
5,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Ativo Realizável a Longo Prazo,2071636.0
6,1.02.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
7,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Títulos Designados a Valor Justo,0.0
8,1.02.01.02,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,1.02.01.03,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0


### <h4>Drop rows</h4>

Select rows with only 3 elements:

In [45]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras,0.0
5,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Ativo Realizável a Longo Prazo,2071636.0


Check if there is at least another row starting with '1.01.01', '1.01.02' or '1.02.01':

In [46]:
for idx in idx_list:
    mask = [True]*len(ds_conta_3)
    mask[idx] = False
    if any(row[:round] == cd_conta_joincol[idx] for row in cd_conta_split[mask]):
        df = df.drop(idx)
    else:
        df.loc[idx, 'CD_CONTA'] = df.loc[idx, 'CD_CONTA'] + '.00'

df.reset_index(inplace=True, drop=True)

In [47]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
2,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos para Negociação,0.0
3,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos Designados a Valor Justo,0.0
4,1.02.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Títulos Designados a Valor Justo,0.0
6,1.02.01.02,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
7,1.02.01.03,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
8,1.02.01.04,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Contas a Receber,124569.0


## <h3>ROUND 4</h3>

In [48]:
round=4

In [49]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
2,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos para Negociação,0.0
3,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos Designados a Valor Justo,0.0
4,1.02.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Títulos Designados a Valor Justo,0.0
6,1.02.01.02,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
7,1.02.01.03,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
8,1.02.01.04,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Contas a Receber,124569.0


### <h4>Join</h4>

In [50]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with only 4 elements:

In [51]:
cd_conta_len = [len(lst) for lst in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [52]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.02.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
6,1.02.01.02,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
7,1.02.01.03,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
8,1.02.01.04,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Contas a Receber,124569.0


Join:

In [53]:
ds_conta_4 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [54]:
ds_conta_4 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [55]:
if pd.Series(ds_conta_4).equals(df.DS_CONTA):
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_4', value=ds_conta_4)

In [56]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
2,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos para Negociação,0.0
3,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
4,1.02.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
6,1.02.01.02,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
7,1.02.01.03,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
8,1.02.01.04,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Contas a Receber,Contas a Receber,124569.0


### <h4>Drop rows</h4>

Select rows with 4 elements:

In [57]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.02.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
6,1.02.01.02,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
7,1.02.01.03,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
8,1.02.01.04,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Contas a Receber,Contas a Receber,124569.0


Check if there is at least another row starting with '1.01.02.01.01', '1.01.02.01.02' or '1.02.01.01.01':

In [58]:
for idx in idx_list:
    mask = [True]*len(ds_conta_4)
    mask[idx] = False
    if any(row[:round] == cd_conta_joincol[idx] for row in cd_conta_split[mask]):
        df = df.drop(idx)
    else:
        df.loc[idx, 'CD_CONTA'] = df.loc[idx, 'CD_CONTA'] + '.00'

df.reset_index(inplace=True, drop=True)

In [59]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos para Negociação,0.0
2,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
3,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
4,1.02.01.02.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.02.01.03.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
6,1.02.01.04.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Contas a Receber,Contas a Receber,124569.0


## <h3>ROUND 5</h3>

In [60]:
round=5

In [61]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos para Negociação,0.0
2,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
3,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
4,1.02.01.02.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.02.01.03.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
6,1.02.01.04.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Contas a Receber,Contas a Receber,124569.0


### <h4>Join</h4>

In [62]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with 5 elements:

In [63]:
cd_conta_len = [len(lst) for lst in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [64]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos para Negociação,0.0
2,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
3,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
4,1.02.01.02.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.02.01.03.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
6,1.02.01.04.00,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Contas a Receber,Contas a Receber,124569.0


Join:

In [66]:
ds_conta_5 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [67]:
ds_conta_5 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [68]:
if pd.Series(ds_conta_5).equals(df.DS_CONTA):
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_5', value=ds_conta_5)

SystemExit: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
